In [8]:
import numpy as np
import pandas as pd

In [23]:
###########################################
#             Preparing Data              #
###########################################

file = open('C:/Users/artur/Downloads/database.csv')
original_file = file.read()

rowsplit_data = original_file.splitlines()
data_examples = np.array([rows.split(',') for rows in rowsplit_data])
data_examples = data_examples[1:]

print('Original')
print('data_examples shape:', data_examples.shape)


##################################
#     Forming 'year' feature     #
##################################

def isdigit(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

years = np.array([year[6:] for year in data_examples[:, 0]])
indicis = []
for i, year in enumerate(years):
    if not isdigit(year):
        indicis.append(i)

data_examples = np.delete(data_examples, indicis, 0)
years = np.delete(years, indicis, 0)

years = np.array([int(year) for year in years])


##################################
#         Clearing Data          #
##################################

unneeded_info = [0, 1, 2, 3, 6, 7, 8, 10, 11, 12, 13, 14, 16]

target = np.array([float(d) for d in data_examples[:, 8]])

latitude = np.array([float(lat) for lat in data_examples[:, 2]])
longitude = np.array([float(lon) for lon in data_examples[:, 3]])

sin_latitude = np.sin(latitude)
sin_longitude = np.sin(longitude)

cos_latitude = np.cos(latitude)
cos_longitude = np.cos(longitude)

data_examples = np.delete(data_examples, unneeded_info, 1)
data_examples = np.column_stack((years, sin_latitude, sin_longitude, cos_latitude, cos_longitude, data_examples))


#################################
#       Using pandas here       #
#################################

data_examples_5 = pd.get_dummies(data_examples[:, 5]).values
data_examples_7 = pd.get_dummies(data_examples[:, 7]).values
data_examples_8 = pd.get_dummies(data_examples[:, 8]).values
data_examples_9 = pd.get_dummies(data_examples[:, 9]).values
data_examples_10 = pd.get_dummies(data_examples[:, 10]).values
data_examples_11 = pd.get_dummies(data_examples[:, 11]).values
data_examples_12 = pd.get_dummies(data_examples[:, 12]).values

unneeded_info = [5, 7, 8, 9, 10, 11, 12]
data_examples = np.delete(data_examples, unneeded_info, 1)
data_examples = np.column_stack((data_examples,
                                 data_examples_5, 
                                 data_examples_7, 
                                 data_examples_8, 
                                 data_examples_9, 
                                 data_examples_10, 
                                 data_examples_11, 
                                 data_examples_12))

print('Cleaned')
print('data_examples shape:', data_examples.shape)

Original
data_examples shape: (23412, 21)
Cleaned
data_examples shape: (23409, 299)


In [26]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [27]:
model = XGBRegressor(nthread=-1)
model.fit(X=data_examples, y=target)

mean_squared_error(target, model.predict(data_examples))

0.14814239781294256